In [1]:
import pandas as pd
import numpy as np
import openpyxl as opxl
import os.path
import pycountry
import pycountry_convert as pc
import datetime as dt
import json
import os.path

In [3]:
def importExcelSheets():
    magento = pd.read_excel('database_sheets/magento_may.xlsx')
    new_magento = pd.read_excel('database_sheets/magento_july.xlsx')
    magento_sept = pd.read_excel('database_sheets/magento_sept.xlsx')
    lot_master = pd.read_excel('database_sheets/lot_master.xlsx', dtype = object)
    prms = pd.read_excel('database_sheets/prms.xlsx')
    unspsc_codes = pd.read_excel('database_sheets/unspsc_codes.xlsx')
    unspsc_codes.columns = unspsc_codes.iloc[0]
    origin = pd.read_excel('database_sheets/country_of_origin.xlsx', dtype=object)
    pd.set_option("max_rows", None)
    pd.set_option("max_columns", None)
    return [magento, new_magento, lot_master, prms, unspsc_codes, origin, magento_sept]

In [4]:
def tidyDescription(desc):
    desc = desc.replace('&TRADE', '')
    desc = desc.replace('Â®', '')
    desc = desc.replace('Î¼', '')
    desc = desc.replace('& Growâ„¢', '')
    desc = desc.replace('â‰', '')
    desc = desc.replace('&Beta', '')
    desc = desc.replace('Â‰¥', '')
    desc = desc.replace('Â', '')
    desc = desc.replace('<em>', '')
    desc = desc.replace('</em>', '')
    desc = desc.replace('®','')
    desc = desc.replace('<F128>','')
    desc = desc.replace('<130>','')
    desc = desc.replace('<F255>','')
    desc = desc.replace('&deg;', '°')
    desc = desc.replace('deg ', '°')
    if desc.endswith(','):
        desc = desc[:len(desc)-1]
    return desc

In [5]:
database_sheets = importExcelSheets()


FileNotFoundError: [Errno 2] No such file or directory: 'MPBIO_Products_Images_20_10_21.xlsx'

In [72]:
fisher = pd.read_excel('forms/fisher_form.xlsx', dtype=object, sheet_name=None, header=None)
# fisher['Regulatory'].columns = np.arange(len(fisher['Regulatory'].columns))
# new_columns = [i.strip() for i in fisher['General Info'].columns]
# fisher['General Info'].columns = new_columns
# fisher['General Info'].columns = np.arange(len(fisher['General Info'].columns))
fisher['General Info']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112
0,Line Errors,Supplier Catalog Number,Item Description - 30 Characters,Item Description - 240 Characters,Item Description - 1000 Characters,Brand,Keyword(1),Keyword(2),Keyword(3),Keyword(4),Keyword(5),Attach Image,Manufacturer Name,Manufacturer Catalog Number,Manufacturer Lead-Time,Standard Unit,Standard Unit Quantity,Unit of Use,Standard Unit Weight,Standard Unit Length,Standard Unit Width,Standard Unit Height,Package Size Quantity,Package Size Unit,Supplier UOM,Conversion from Supplier to Fisher,Alternate Unit,Alternate Unit Quantity,Alternate Unit Weight,Alternate Unit Length,Alternate Unit Width,Alternate Unit Height,Standard Unit Cost,Standard Unit List,Alternate Unit List,Discount %,Pricing Expiration Date,Customer Specific Item,Customer Specific - Customer Name,Customer Specific - Customer Account,Recommended Inside Delivery,UNSPSC Code,Country of Origin,Hazardous,UN/NA#,Hazard Class,Proper Shipping Name (PSN),Packing Group,Health Industry Bar Code,Units on a Pallet,Pallet layers,Build to Stock or Build to Order,Serial / Lot Control,Maximum Lot Batch Size,Shelf Life,Shelf Life (Days),Source Receiving Days,Source Expiration Days,Expiration,Product Info on Shipping Box,Lot Info on Shipping Box,Expiration Info on Shipping Box,Minimum Order Quantity,Certificates Available,Certificates for Lot Controlled Products,Certificate of Analysis,Certificate of Conformation,Certificate of Sterility,Certificate of Validation,Animal Source of Origin,TSE/BSE,Certificate of Origin,Certificate of Guarantee,Certificate of Quality,Certificate of Processing,Letter of Certification,Certficate of Irradiation,Warranty Period,Installation,Volts,HZ,Phase,Replace Catalog Number,Accessory,Accessory Type,Competitor 1 Name,Competitor 1 Catalog Number,Competitor 1 Equivalency,Competitor 1 Comments,Competitor 2 Name,Competitor 2 Catalog Number,Competitor 2 Equivalency,Competitor 2 Comments,Green Product,Environmentally Friendly,Energy Star,Green Seal of Approval,BioBased,FEMP Energy Efficient Item,U.S. EPA Significant New Alternatives Program,Ozone Free,Non-Toxic,Recyclable Waste,"Manufactured with recycled, natural or renewab...",Comprehensive Procurement,PRIME,NESHAP,EPEAT,Canada Availability,Currency,Canada Standard Unit Cost,Canada Standard Unit List,Canada Alternate Unit List
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
def fillFisher_Old(filename, magento, new_magento, lot_master, prms, unspsc_codes, origin, magento_sept):
    fisher_file = pd.ExcelFile('forms/fisher_form.xlsx')
    fisher = pd.read_excel(fisher_file, 'General Info', dtype=object)
    regulatory = pd.read_excel(fisher_file, 'Regulatory', dtype=object)
    new_columns = [i.strip() for i in regulatory.columns]
    regulatory.columns = new_columns
    new_columns = [i.strip() for i in fisher.columns]
    fisher.columns = new_columns
    wb = opxl.load_workbook(filename)
    skus = wb.active
    for i in range(2, skus.max_row+1):
        regulatory.loc[i-2, 'Supplier Catalog Number'] = str(skus['A'+str(i)].value)
    for i in range(2, skus.max_row+1):
        fisher.loc[i-2, 'Supplier Catalog Number'] = str(skus['A'+str(i)].value)

    for i in range(len(regulatory)):
        sku = regulatory['Supplier Catalog Number'][i]
        product_info = magento.loc[magento['sku'] == sku]
        lot_info = lot_master.loc[lot_master['Product number'] == sku]
        prms_info = prms.loc[prms['SKU'] == sku]
        unspsc_info = unspsc_codes.loc[unspsc_codes['Part Number'] == sku]
        product_info_july = new_magento.loc[new_magento['sku'] == sku]
        product_info_sept = magento_sept.loc[magento_sept['sku'] == sku]     

        if not product_info_july.empty:
            tariff_code = product_info_july['tariff_code'].values[0]
        else:
            tariff_code = ''
        if not product_info.empty:
            if not product_info_sept.empty:
                cas_number = product_info_sept['cas_number'].values[0]
                pack_size = product_info_sept['pack_size_joined'].values[0]
            else:
                cas_number = product_info['cas_number'].values[0]
                pack_size = product_info['pack_size_joined'].values[0]

            if not prms_info.empty:
                price = prms_info['USD List Price'].values[0]
                hazard_statements = prms_info['UN#'].values[0]
                packing_group = prms_info['Packing Group'].values[0]
                alt_storage_temp = prms_info['Storage Temp'].values[0]
                ship_temp = prms_info['Ship Temp'].values[0]
                country_of_origin = prms_info['Country of Origin (most recent lot)'].values[0]
                cas_number = product_info['cas_number'].values[0]
                storage_temp = prms_info['Storage Temp'].values[0]
                
            else:
                price = product_info['price'].values[0]
                packing_group = product_info['packing_group'].values[0]
                ship_temp = product_info['ship_conditions'].values[0]
                hazard_statements = product_info['hazard_statements'].values[0]
                storage_temp = ''
                
            if not unspsc_info.empty:
                unspsc = unspsc_info['UNSPSC'].values[0]
            else:
                unspsc = product_info['unspsc'].values[0]
            
            short_desc = product_info['short_description'].values[0]

            quantity = product_info['lk_packaging_facet'].values[0]
            host = product_info['host'].values[0]
            msds_avail = skus['B'+str(i+1)].value
            DOT_PSN = 'N/A'
            hazard_class = product_info['hazard_class'].values[0]
            biochem_physiol_actions = product_info['biochem_physiol_actions'].values[0]
            storage_and_handling = product_info['storage_and_handling'].values[0]
            group_name = product_info['prms_group_name'].values[0]
            img_link = product_info['base_image'].values[0]
            keywords = product_info['meta_keywords'].values[0]

            if type(pack_size) == str:
                unit = ''.join([i for i in pack_size if not i.isdigit()])
            else:
                unit = ''
            
            if sku.startswith('07') or sku.startswith('08') or sku.startswith('09') or sku.startswith('11'):
                regulatory['Primary Commodity Code'][i] = 'Diagnostics'
            elif type(cas_number) == str and len(cas_number) > 0:
                regulatory['Primary Commodity Code'][i] = 'Chemicals'
            elif 'preps' in unit or 'Preps' in unit:
                regulatory['Primary Commodity Code'][i] = 'Consumables'
            elif sku.startswith('02') or sku.startswith('03') or sku.startswith('04') or sku.startswith('05'):
                regulatory['Primary Commodity Code'][i] = 'Diagnostics'

            regulatory['Safety Data Sheet Code'][i] = 99998
            
            if type(storage_temp) == str:
                if storage_temp == 'AM':
                    regulatory['Storage Code'][i] = 'GWN4'
                elif storage_temp == 'FR' or storage_temp == '70' or storage_temp == '80':
                    regulatory['Storage Code'][i] = 'DFD1'
                elif storage_temp == 'RF':
                    regulatory['Storage Code'][i] = 'RFC2'
                else:
                    regulatory['Storage Code'][i] = storage_temp
            else:
                regulatory['Storage Code'][i] = 'GWN4'
            
            regulatory['STERILE'][i] = 'N'
            regulatory['Proposition 65'][i] = 'N'
            regulatory['Latex'][i] = 'N'
            regulatory['Lithium Battery'][i] = 'N'
            regulatory['Medical Device'][i] = 'None'
            regulatory['UPC - Standard'][i] = 'NA'
            
            tariff_code = str(tariff_code).replace('.', '')
            if len(tariff_code) >= 4:
                regulatory['Harmonized Tariff Schedule Code'][i] = tariff_code[:4] + '999999'
                
            regulatory['REACH Compliant'][i] = 'N'
            regulatory['RoHS Compliant'][i] = 'N'
            regulatory['Mercury'][i] = 'N'
            regulatory['Benzene %'][i] = 'N'
            regulatory['Asbestos'][i] = 'N'
            regulatory['Iodine'][i] = 'N'
            regulatory['Radioactive Materials'][i] = 'N'
            regulatory['Pesticides'][i] = 'N'
            regulatory['Ethyl Alcohol'][i] = 'N'
            regulatory['DEA List 1 Chemical or Drug'][i] = 'N'
            regulatory['WHMIS Regulated?\n(Y/N)'][i] = 'N'
            regulatory['Marine Pollutant'][i] = 'N'
            regulatory['Is USMCA/CUSMA Certificate available?'][i] = 'N'
            regulatory["Federal  or Int'l Regulations"][i] = 'NA'
                
            if type(host) == str and len(host) > 0:
                regulatory['Animal or Human Origin'][i] = 'Y'
            else:
                regulatory['Animal or Human Origin'][i] = 'N'
            
            regulatory['MDL Number'][i] = 'NA'

            if type(cas_number) == str and not cas_number == 'Not applicable':
                regulatory['Chemical Abstracts Number (CAS 1)'][i] = cas_number
                regulatory['Chemical Abstracts Percentage(1)'][i] = 100

    for i in range(len(fisher)):
        sku = fisher['Supplier Catalog Number'][i]
        product_info = magento.loc[magento['sku'] == sku]
        lot_info = lot_master.loc[lot_master['Product number'] == sku]
        prms_info = prms.loc[prms['SKU'] == sku]
        unspsc_info = unspsc_codes.loc[unspsc_codes['Part Number'] == sku]
        product_info_sept = magento_sept.loc[magento_sept['sku'] == sku]


        if not product_info.empty:
            if not product_info_sept.empty:
                name = product_info_sept['name'].values[0].upper()
                pack_size = product_info_sept['pack_size_joined'].values[0]
            else:
                name = product_info['name'].values[0].upper()
                pack_size = product_info['pack_size_joined'].values[0]


            if not prms_info.empty:
                price = prms_info['USD List Price'].values[0]
                hazard_statements = prms_info['UN#'].values[0]
                packing_group = prms_info['Packing Group'].values[0]
                alt_storage_temp = prms_info['Storage Temp'].values[0]
                ship_temp = prms_info['Ship Temp'].values[0]
                country_of_origin = prms_info['Country of Origin (most recent lot)'].values[0]
                if type(country_of_origin) == str and len(country_of_origin) > 0:
                    country_of_origin = pc.country_alpha3_to_country_alpha2(country_of_origin)
                    fisher['Canada Availability'][i] = 'Y'
                    fisher['Currency'][i] = 'USD'
                else:
                    fisher['Canada Availability'][i] = 'N'
                
            else:
                price = product_info['price'].values[0]
                packing_group = product_info['packing_group'].values[0]
                ship_temp = product_info['ship_conditions'].values[0]
                hazard_statements = product_info['hazard_statements'].values[0]
                
            if not unspsc_info.empty:
                unspsc = unspsc_info['UNSPSC'].values[0]
            else:
                unspsc = product_info['unspsc'].values[0]
            
            short_desc = product_info['short_description'].values[0]
            img_link = product_info['base_image'].values[0]
            quantity = product_info['lk_packaging_facet'].values[0]
            host = product_info['host'].values[0]
            tariff_code = product_info['tariff_code'].values[0]
            cas_number = product_info['cas_number'].values[0]
            DOT_PSN = 'NA'
            hazard_class = product_info['hazard_class'].values[0]
            biochem_physiol_actions = product_info['biochem_physiol_actions'].values[0]
            storage_and_handling = product_info['storage_and_handling'].values[0]
            group_name = product_info['prms_group_name'].values[0]
            img_link = product_info['base_image'].values[0]
            keywords = product_info['meta_keywords'].values[0]
            storage_temp = product_info['storage_and_handling'].values[0]
            if not new_magento.loc[new_magento['sku'] == sku].empty:
                weight_in_lb = new_magento.loc[new_magento['sku'] == sku]['weight'].values[0]
                
                fisher['Standard Unit Weight'][i] = weight_in_lb
            
                if weight_in_lb < 0.5:
                    fisher['Standard Unit Length'][i] = 7
                    fisher['Standard Unit Width'][i] = 4
                    fisher['Standard Unit Height'][i] = 5
                    fisher['Units on a Pallet'][i] = 60
                    fisher['Pallet layers'][i] = 36
                elif weight_in_lb <= 1:
                    fisher['Standard Unit Length'][i] = 12
                    fisher['Standard Unit Width'][i] = 7
                    fisher['Standard Unit Height'][i] = 5
                    fisher['Units on a Pallet'][i] = 18
                    fisher['Pallet layers'][i] = 36
                else:
                    fisher['Standard Unit Length'][i] = 12
                    fisher['Standard Unit Width'][i] = 12
                    fisher['Standard Unit Height'][i] = 12
                    fisher['Units on a Pallet'][i] = 12
                    fisher['Pallet layers'][i] = 15
            

            if type(name) == str:
                name = tidyDescription(name)
                temp_name = ''
                for c in name:
                    if c.isalpha() or c.isnumeric() or c == ' ':
                        temp_name = temp_name + c
                name = temp_name
                if len(name) <= 30:
                    fisher['Item Description - 30 Characters'][i] = name
                else:
                    fisher['Item Description - 30 Characters'][i] = name[:30]
                
                if len(name) <= 240:
                    fisher['Item Description - 240 Characters'][i] = name
                else:
                    fisher['Item Description - 240 Characters'][i] = name[:240]
                    
            fisher['Standard Unit'][i] = 'EA'
            fisher['Standard Unit Quantity'][i] = 1
            fisher['Unit of Use'][i] = 'EA'

            if type(pack_size) == str:
                quant = ''.join([i for i in pack_size if i.isdigit()])
                unit = ''.join([i for i in pack_size if not i.isdigit()])
                fisher['Package Size Quantity'][i] = quant
                if 'preps' in unit or 'Preps' in unit:
                    unit = 'PP'
                elif 'mL' in unit or 'ml' in unit:
                    unit = 'ML'
                elif 'lb' in unit:
                    unit = 'LB'
                elif 'tests' in unit or 'Tests' in unit:
                    unit = 'TS'
                elif 'KU' in unit or 'ku' in unit:
                    unit = 'KU'
                elif 'mg' in unit:
                    unit = 'MG'
                elif 'U' in unit or 'wells' in unit or 'Bags' in unit or 'Bottle' in unit or 'Each' in unit or 'Kit' in unit or 'mCI' in unit:
                    unit = 'UN'
                elif unit == 'g':
                    unit = 'GR'
                elif unit == 'l' or unit == 'L' or 'Liter' in unit:
                    unit = 'LT'
                else:
                    unit = 'UN'
                fisher['Package Size Unit'][i] = unit
            
            if type(price) != str:
                fisher['Standard Unit Cost'][i] = price*0.7
                fisher['Standard Unit List'][i] = price
            
            fisher['Discount %'][i] = '30'
            fisher['Pricing Expiration Date'][i] = '31/DEC/2020'
            
            fisher['UNSPSC Code'][i] = unspsc
            
            if type(hazard_statements) == str and len(hazard_statements) > 0:
                fisher['Hazardous'][i] = 'Y'
            else:
                fisher['Hazardous'][i] = 'N'
            
            fisher['UN/NA#'][i] = hazard_statements
            fisher['Hazard Class'][i] = hazard_class
            fisher['Packing Group'][i] = packing_group

            fisher['Shelf Life'][i] = 'N'
            fisher['Shelf Life  (Days)'][i] = 0
            
            if type(keywords) == str:
                keywords = keywords.split(',')
                for j in range(len(keywords)):
                    if len(keywords[j]) > 11:
                        keywords[j] = keywords[j][:11]
                keywords = list(set(keywords))
                j = 0
                while j<5 and j<len(keywords):
                    fisher.iloc[i, j+6] = keywords[j]
                    fisher.iloc[i, j+6] = keywords[j]
                    j = j+1
                    
            fisher['Manufacturer Lead-Time'][i] = '25'
            fisher['Green Product'][i] = 'N'
            fisher['Country of Origin'][i] = country_of_origin
            fisher['Build to Stock or Build to Order'][i] = 'Build to Order'
            fisher['Serial / Lot Control'][i] = 'NA'
            fisher['Expiration'][i] = 'Y'
            fisher['Product Info on Shipping Box'][i] = 'N'
            fisher['Expiration Info on Shipping Box'][i] = 'Y'
            fisher['Minimum Order Quantity'][i] = 1
            fisher['Certificates Available'][i] = 'Y'
            fisher['Certificate of Analysis'][i] = 'Y'
            fisher['Installation'][i] = 'N'
            
            
    new_fisher = opxl.load_workbook('forms/fisher_form.xlsx')
    gen_sheet = new_fisher['General Info']
    regulatory_sheet = new_fisher['Regulatory']
    if gen_sheet.max_row < skus.max_row:
        for j in range(skus.max_row - gen_sheet.max_row+1):
            gen_sheet.insert_rows(gen_sheet.max_row-1)
    if regulatory_sheet.max_row < skus.max_row:
        regulatory_sheet.append([''])
        for j in range(skus.max_row - regulatory_sheet.max_row + 1):
            regulatory_sheet.insert_rows(regulatory_sheet.max_row)
    i = 0
    for row in gen_sheet.iter_rows(min_row=2):
        if i < len(fisher):
            for j in range(len(fisher.columns)-1):
                if not fisher.iloc[i, j] == 'None':
                    row[j].value = fisher.iloc[i, j]
        else:
            break
        i = i+1
    i = 0
    for row in regulatory_sheet.iter_rows(min_row=2):
        if i < len(regulatory):
            for j in range(len(regulatory.columns)-1):
                if not fisher.iloc[i, j] == 'None':
                    row[j].value = regulatory.iloc[i, j]
        else:
            break
        i = i+1

    new_fisher.save('outputs/old_product_outputs/old_fisher_output.xlsx')
    return fisher
    # new_fisher.save('../../outputs/old_product_outputs/old_fisher_output.xlsx')

In [79]:
fisher = pd.read_excel('forms/fisher_form.xlsx', dtype=object, sheet_name=None)
# fisher['Regulatory'].columns = np.arange(len(fisher['Regulatory'].columns))
# new_columns = [i.strip() for i in fisher['General Info'].columns]
# fisher['General Info'].columns = new_columns
# fisher['General Info'].columns = np.arange(len(fisher['General Info'].columns))
for i in fisher['General Info'].columns: print(i)

Line Errors
Supplier Catalog Number
Item Description - 30 Characters
Item Description - 240 Characters
Item Description - 1000 Characters
Brand
Keyword(1) 
Keyword(2) 
Keyword(3) 
Keyword(4) 
Keyword(5) 
Attach Image
Manufacturer Name
Manufacturer Catalog Number
Manufacturer Lead-Time
Standard Unit
Standard Unit Quantity
Unit of Use
Standard Unit Weight
Standard Unit Length
Standard Unit Width
Standard Unit Height 
Package Size Quantity
Package Size Unit
Supplier UOM
Conversion from Supplier to Fisher
Alternate Unit 
Alternate Unit Quantity 
Alternate Unit Weight 
Alternate Unit Length 
Alternate Unit Width 
Alternate Unit Height 
Standard Unit Cost
Standard Unit List
Alternate Unit List 
Discount %
Pricing Expiration Date
Customer Specific Item
Customer Specific - Customer Name
Customer Specific - Customer Account
Recommended Inside Delivery
UNSPSC Code 
Country of Origin
Hazardous 
UN/NA# 
Hazard Class 
Proper Shipping Name (PSN) 
Packing Group 
Health Industry Bar Code 
Units on a P

In [45]:
print(len(vwr.columns))
# vwr.iloc[1, 179]

180


' DEA Establishment Registration Number'

In [87]:
new_fisher = fillFisher_Old('input_sheets/VWR testing.xlsx', database_sheets[0], database_sheets[1], database_sheets[2], database_sheets[3], database_sheets[4], database_sheets[5], database_sheets[6])

In [67]:
new_regulatory

,Line Errors,Supplier Catalog Number,Item Description - 30 Characters,Primary Commodity Code,Safety Data Sheet Code,Attach SDS,Storage Code,Recommended Inside Delivery,STERILE,Proposition 65,Proposition 65 Detail,Latex,Lithium Battery,Attach Lithium Battery Test Summary Report,Medical Device,UPC - Standard,UPC - Alternate,Canadian Medical Device Class,Health Canada License #,Medical Device Insert/Attachment,Prescription Medical Device,FDA 510k number,FDA product code,GTIN - EA,GTIN - Pack,GTIN - Case,Prescription Drug,Over the Counter Drug,National Drug Code,Harmonized Tariff Schedule Code,REACH Compliant,RoHS Compliant,Mercury,Milligrams of Mercury,Benzene,Benzene %,Asbestos,Iodine,Iodine %,Radioactive Materials,Pesticides,What is the EPA registration number,CA Dept. of Pesticides Regulation Number,Ethyl Alcohol,Animal or Human Origin,Animal or Human Origin Types,Supplier Website Item Link,MDL Number,Chemical Abstracts Number (CAS 1),Chemical Abstracts Percentage(1),Chemical Abstracts Number (CAS 2),Chemical Abstracts Percentage(2),Chemical Abstracts Number (CAS 3),Chemical Abstracts Percentage(3),Chemical Abstracts Number (CAS 4),Chemical Abstracts Percentage(4),Chemical Abstracts Number (CAS 5),Chemical Abstracts Percentage(5),DEA List 1 Chemical or Drug,Hydro,UN Number,WHMIS Regulated?\n(Y/N),Marine Pollutant,Is USMCA/CUSMA Certificate available?,Federal or Int'l Regulations
0,NaN,0210000801,NaN,Chemicals,99998,NaN,DFD1,NaN,N,N,NaN,N,N,NaN,None,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,NaN,NaN,N,N,N,NaN,N,N,NaN,NaN,N,N,NaN,NaN,NA,51963-61-2,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N,N,N,NA
1,NaN,0210000805,NaN,Chemicals,99998,NaN,DFD1,NaN,N,N,NaN,N,N,NaN,None,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,NaN,NaN,N,N,N,NaN,N,N,NaN,NaN,N,N,NaN,NaN,NA,51963-61-2,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N,N,N,NA
2,NaN,0210000810,NaN,Chemicals,99998,NaN,DFD1,NaN,N,N,NaN,N,N,NaN,None,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,NaN,NaN,N,N,N,NaN,N,N,NaN,NaN,N,N,NaN,NaN,NA,51963-61-2,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N,N,N,NA
3,NaN,02100109.2,NaN,Chemicals,99998,NaN,DFD1,NaN,N,N,NaN,N,N,NaN,None,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2940999999,N,N,N,NaN,NaN,N,N,N,NaN,N,N,NaN,NaN,N,N,NaN,NaN,NA,4773-29-9,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N,N,N,NA
4,NaN,0210019125,NaN,Chemicals,99998,NaN,GWN4,NaN,N,N,NaN,N,N,NaN,None,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2933999999,N,N,N,NaN,NaN,N,N,N,NaN,N,N,NaN,NaN,N,N,NaN,NaN,NA,6055-72-7,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N,N,N,NA
5,NaN,0210042780,NaN,Chemicals,99998,NaN,GWN4,NaN,N,N,NaN,N,N,NaN,None,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2842999999,N,N,N,NaN,NaN,N,N,N,NaN,N,N,NaN,NaN,N,N,NaN,NaN,NA,7790-58-1,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N,N,N,NA
6,NaN,0210082410,NaN,Chemicals,99998,NaN,GWN4,NaN,N,N,NaN,N,N,NaN,None,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2918999999,N,N,N,NaN,NaN,N,N,N,NaN,N,N,NaN,NaN,N,N,NaN,NaN,NA,867-55-0,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N,N,N,NA
7,NaN,0210090450,NaN,Chemicals,99998,NaN,DFD1,NaN,N,N,NaN,N,N,NaN,None,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2934999999,N,N,N,NaN,NaN,N,N,N,NaN,N,N,NaN,NaN,N,N,NaN,NaN,NA,1716-06-9,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N,N,N,NA
8,NaN,0210137980,NaN,Chemicals,99998,NaN,GWN4,NaN,N,N,NaN,N,N,NaN,None,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,NaN,NaN,N,N,N,NaN,N,N,NaN,NaN,N,N,NaN,NaN,NA,1510-21-0,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N,N,N,NA
9,NaN,0210192780,NaN,Chemicals,99998,NaN,GWN4,NaN,N,N,NaN,N,N,NaN,None,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2933999999,N,N,N,NaN,NaN,N,N,N,NaN,N,N,NaN,NaN,N,N,NaN,NaN,NA,16052-06-5,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N,N,N,NA
